# Connecting to Drive

In [1]:
# Connecting to google drive
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Path
%cd "/content/drive/MyDrive/Bridge construction health/brridge-failure-detection"

/content/drive/MyDrive/Bridge construction health/brridge-failure-detection


In [9]:
! ls -ltrh

total 19M
-rw------- 1 root root 868K Aug 19 15:51 data.rar
-rw------- 1 root root 3.4M Aug 19 15:51 data1.rar
-rw------- 1 root root 373K Aug 21 16:26 wavelet.ipynb
-rw------- 1 root root 3.0K Aug 21 16:26 sst_nino3.dat
drwx------ 2 root root 4.0K Aug 21 16:35 data
drwx------ 2 root root 4.0K Aug 21 16:36 data1
drwx------ 2 root root 4.0K Aug 23 12:01 main_datatset
-rw------- 1 root root  50K Aug 30 18:29 bridge_failure_detection_mixed.ipynb
-rw------- 1 root root  43K Aug 30 19:26 analysis.ipynb
-rw------- 1 root root 9.5K Aug 30 19:44 github_handler.ipynb
drwx------ 5 root root 4.0K Sep  9 15:13 sensor1
-rw------- 1 root root  14M Sep 11 15:54 sensor1.rar
-rw------- 1 root root  55K Sep 11 16:38 bridge_failure_detection_one_sensor.ipynb


# Imports and installations

In [4]:
import os
import shutil
from pathlib import Path

In [5]:
from PIL import Image
import numpy as np

# Unzipping

In [ ]:
! unrar x "./sensor1.rar"

# Vectorization of images

In [10]:
def vectorize_rgb_image_to_array(png_path):
    # Load the PNG image
    image = Image.open(png_path)

    # Convert the image to RGB mode (if it's not already)
    image = image.convert('RGB')

    # Convert the image to a numpy array
    img_array = np.array(image)

    return img_array

In [11]:
! ls "./sensor1/After_reconstruction" | head

spectrogram_DataMidel_Day100_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day101_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day102_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day103_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day104_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day105_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day106_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day107_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day108_Sensor_1.mat_col1.png
spectrogram_DataMidel_Day109_Sensor_1.mat_col1.png


## Checking the sample number of each class

In [15]:
len(os.listdir("./sensor1/When_rebuilding"))

245

In [16]:
len(os.listdir("./sensor1/Before_renovation"))

425

In [17]:
len(os.listdir("./sensor1/After_reconstruction"))

211

## Vetorizing all

In [21]:
X = []
y = []

In [18]:
all_classes = os.listdir("./sensor1/")

In [19]:
all_classes_dict = {
    "Before_renovation": 0,
    "After_reconstruction": 1,
    "When_rebuilding": 2
}

In [22]:
# Vectorizing and added labels of images
for lbl_class in all_classes:
  print(lbl_class)
  images = os.listdir(f"./sensor1/{lbl_class}")

  for img in images:
    img_path = f"./sensor1/{lbl_class}/{img}"
    img_array = vectorize_rgb_image_to_array(img_path)
    X.append(img_array)
    y.append(all_classes_dict[lbl_class])

After_reconstruction
Before_renovation
When_rebuilding


# Model Architecture

In [23]:
from sklearn.model_selection import train_test_split

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [24]:
X_train.shape

(704, 240, 240, 3)

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(240, 240, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)                    │ (None, 238, 238, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 119, 119, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 117, 117, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 58, 58, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 215296)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │      13,779,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,798,595 (52.64 MB)

 Trainable params: 13,798,595 (52.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(240, 240, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(240, 240, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)                    │ (None, 238, 238, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 119, 119, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 117, 117, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 58, 58, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 56, 56, 32)          │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 28, 28, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │       1,605,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,663,075 (6.34 MB)

 Trainable params: 1,663,075 (6.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 149s 7s/step - accuracy: 0.3415 - loss: 37.2111 - val_accuracy: 0.4802 - val_loss: 1.0733
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 0.5022 - loss: 1.0766 - val_accuracy: 0.4859 - val_loss: 1.0504
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 132s 6s/step - accuracy: 0.4841 - loss: 1.0399 - val_accuracy: 0.4859 - val_loss: 1.0135
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.4929 - loss: 0.9990 - val_accuracy: 0.5028 - val_loss: 0.9514
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 124s 6s/step - accuracy: 0.5222 - loss: 0.9456 - val_accuracy: 0.5876 - val_loss: 0.9025
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 137s 6s/step - accuracy: 0.5987 - loss: 0.8855 - val_accuracy: 0.4181 - val_loss: 1.0847
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 128s 6s/step - accuracy: 0.4721 - loss: 1.0767 - val_accuracy: 0.4915 - val_loss: 1.0563
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 144s 6s/step - accuracy: 0.5536 - loss: 1.0196 - val_accuracy: 0.5198 - 

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.4188 - loss: 3.3138
Test loss: 3.155606985092163
Test accuracy: 0.4406779706478119
